In [1]:
!pip install music21

In [2]:
from music21 import converter,note,chord,stream,instrument
import glob
import pickle
import numpy as np
from keras.utils import np_utils

TensorFlow 1.x selected.


Using TensorFlow backend.


## Reading and processing the music files

In [0]:
midi_song = converter.parse('./drive/My Drive/Datasets/midi_songs/EyesOnMePiano.mid')

In [6]:
midi_song.show('midi')

In [0]:
#midi_song.show('text')

In [8]:
elements_to_parse = midi_song.flat.notes
print(type(elements_to_parse))
print(len(elements_to_parse))

<class 'music21.stream.iterator.StreamIterator'>
1421


In [0]:
#for e in elements_to_parse:
    #print(e,e.offset)

In [10]:
print(elements_to_parse[3])
elements_to_parse[3].pitch

<music21.note.Note A>


<music21.pitch.Pitch A6>

In [11]:
print(elements_to_parse[68])
print(elements_to_parse[68].normalOrder)

<music21.chord.Chord F#4 C#5>
[1, 6]


In [12]:
'+'.join(str(e) for e in elements_to_parse[68].normalOrder)

'1+6'

In [0]:
notes_demo = []

for ele in elements_to_parse:
    # if the element is note, then store its pitch
    if isinstance(ele,note.Note):
        notes_demo.append(str(ele.pitch))
    # if the element is a chord, then split the chord into notes and then join them with any operator
    elif isinstance(ele,chord.Chord):
        notes_demo.append('+'.join(str(e) for e in ele.normalOrder))

#print(notes_demo)

In [14]:
notes = []
for file in glob.glob('./drive/My Drive/Datasets/midi_songs/*.mid'):
    midi = converter.parse(file) 
    elements_to_parse = midi.flat.notes
    print("parsing %s"%file)
    for ele in elements_to_parse:
        # if the element is note, then store its pitch
        if isinstance(ele,note.Note):
            notes.append(str(ele.pitch))
        # if the element is a chord, then split the chord into notes and then join them with any operator
        elif isinstance(ele,chord.Chord):
            notes.append('+'.join(str(e) for e in ele.normalOrder))
#print(notes)

parsing ./drive/My Drive/Datasets/midi_songs/8.mid
parsing ./drive/My Drive/Datasets/midi_songs/AT.mid
parsing ./drive/My Drive/Datasets/midi_songs/BlueStone_LastDungeon.mid
parsing ./drive/My Drive/Datasets/midi_songs/0fithos.mid
parsing ./drive/My Drive/Datasets/midi_songs/Still_Alive-1.mid
parsing ./drive/My Drive/Datasets/midi_songs/FFIX_Piano.mid
parsing ./drive/My Drive/Datasets/midi_songs/Fyw_piano.mid
parsing ./drive/My Drive/Datasets/midi_songs/balamb.mid
parsing ./drive/My Drive/Datasets/midi_songs/Ff7-Jenova_Absolute.mid
parsing ./drive/My Drive/Datasets/midi_songs/Kingdom_Hearts_Traverse_Town.mid
parsing ./drive/My Drive/Datasets/midi_songs/FFX_-_Ending_Theme_(Piano_Version)_-_by_Angel_FF.mid
parsing ./drive/My Drive/Datasets/midi_songs/ff4-fight1.mid
parsing ./drive/My Drive/Datasets/midi_songs/decisive.mid
parsing ./drive/My Drive/Datasets/midi_songs/FF4.mid
parsing ./drive/My Drive/Datasets/midi_songs/FF8_Shuffle_or_boogie_pc.mid
parsing ./drive/My Drive/Datasets/midi_so

In [15]:
n_vocab = len(set(notes))
print(len(notes))
print(n_vocab)

65730
381


In [16]:
print("Total len of vocab %d \nUnique elements of vocab %d"%(len(notes),n_vocab))

Total len of vocab 65730 
Unique elements of vocab 381


In [0]:
import os
import pickle

In [0]:
with open("./drive/My Drive/PreprocessedData/notes",'wb') as f:
    pickle.dump(notes,f)
f.close()

In [0]:
with open('./drive/My Drive/PreprocessedData/notes','rb') as f:
    note_load = pickle.load(f)
f.close()

In [20]:
n_vocab = len(set(notes))
print(len(notes))
print(n_vocab)

65730
381


## Generating the sequential data for the LSTM model

In [0]:
notes = note_load

In [0]:
snotes = sorted(set(notes))

In [0]:
ele_int = dict((ele,idx) for idx,ele in enumerate(snotes))

In [24]:
ele_int["F#2"]
n = np.array(notes)
n[n=="F#4"].shape
#n.shape

(868,)

In [0]:
sequence_length = 50

network_input = []
network_output = []

for i in range(len(notes) - sequence_length):
    seq_in = notes[i:i+sequence_length] # in the string format
    seq_out = notes[i+sequence_length]
    
    network_input.append([ele_int[ch] for ch in seq_in])
    network_output.append(ele_int[seq_out])

In [26]:
print(len(network_input))
n_patterns = len(network_input)
print(network_input[0])

65680
[333, 352, 330, 333, 352, 330, 378, 378, 333, 333, 340, 66, 308, 340, 66, 308, 366, 366, 311, 311, 367, 275, 363, 367, 275, 363, 304, 275, 363, 304, 275, 363, 379, 83, 375, 379, 83, 375, 353, 333, 330, 353, 333, 330, 330, 330, 378, 378, 333, 333]


In [27]:
print(len(network_output))

65680


In [28]:
network_input = np.reshape(network_input,(n_patterns,sequence_length,1))
print(network_input.shape)

(65680, 50, 1)


In [0]:
# Normalize our data [0,1]
normalized_network_input = network_input/float(n_vocab)
#print(normalized_network_input)

In [30]:
# Converting the y values into one hot vectors
network_output = np_utils.to_categorical(network_output)
print(network_output.shape)

(65680, 381)


In [31]:
print(normalized_network_input.shape)
print(network_output.shape)

(65680, 50, 1)
(65680, 381)


## Creating the LSTM model

In [0]:
from keras.models import Sequential,load_model
from keras.layers import *
from keras.callbacks import ModelCheckpoint,EarlyStopping
from sklearn.metrics import classification_report,confusion_matrix
from keras.optimizers import Adam

In [0]:
adam = Adam(lr = 1e-3)

In [110]:
model = Sequential()
model.add(LSTM(512,input_shape = (normalized_network_input.shape[1],normalized_network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(512,activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(n_vocab,activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy',optimizer = adam)
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 50, 512)           1052672   
_________________________________________________________________
dropout_7 (Dropout)          (None, 50, 512)           0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 50, 512)           2099200   
_________________________________________________________________
dropout_8 (Dropout)          (None, 50, 512)           0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_5 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)              

In [111]:
checkpoint = ModelCheckpoint("model.hdf5",monitor='loss',verbose=0,save_best_only=True,mode = 'min')

hist = model.fit(normalized_network_input,network_output,epochs = 100,batch_size = 1024,callbacks = [checkpoint])

Epoch 1/100
65680/65680 [==============================] - 57s 866us/step - loss: 4.8638
Epoch 2/100
65680/65680 [==============================] - 56s 850us/step - loss: 4.7411
Epoch 3/100
65680/65680 [==============================] - 56s 853us/step - loss: 4.7314
Epoch 4/100
65680/65680 [==============================] - 56s 853us/step - loss: 4.7259
Epoch 5/100
65680/65680 [==============================] - 56s 855us/step - loss: 4.7220
Epoch 6/100
65680/65680 [==============================] - 56s 853us/step - loss: 4.7201
Epoch 7/100
65680/65680 [==============================] - 56s 854us/step - loss: 4.6885
Epoch 8/100
65680/65680 [==============================] - 56s 852us/step - loss: 4.5849
Epoch 9/100
65680/65680 [==============================] - 56s 853us/step - loss: 4.5441
Epoch 10/100
65680/65680 [==============================] - 56s 854us/step - loss: 4.5266
Epoch 11/100
65680/65680 [==============================] - 56s 853us/step - loss: 4.5081
Epoch 12/100
65680/

In [0]:
model.save("model.hdf5")

## Making predictions on the model

In [0]:
model = load_model("model.hdf5")

In [0]:
sequence_length = 50
network_input = []
for i in range(len(notes) - sequence_length):
    seq_in = notes[i:i+sequence_length] # in the string format
    network_input.append([ele_int[ch] for ch in seq_in])

In [0]:
# Taking a random start sequence to generate the next sequence
start = np.random.randint(len(network_input))
pattern = network_input[start]
prediction_output = []
int_to_ele = dict((idx,ele)for idx,ele in enumerate(snotes))

for i in range(200):
    prediction_input = np.reshape(pattern,(1,sequence_length,1))
    prediction_input = prediction_input/float(n_vocab)
    
    prediction = model.predict(prediction_input)
    idx = np.argmax(prediction)
    prediction_output.append(int_to_ele[idx])
    
    pattern.append(idx)
    pattern = pattern[1:]    

In [114]:
print(prediction_output)

['A4', '0+5', 'C5', 'A4', 'E5', 'C5', 'B4', 'D5', 'E5', '4+9', 'C5', 'A4', '0+5', '4+9', 'C5', 'A4', 'F#5', 'C5', 'A4', '0+5', 'C5', 'A4', 'E5', 'E3', 'C5', 'B2', 'B4', 'C3', 'D5', 'G#2', 'E5', '4+9', 'C5', 'A4', '0+5', '4+9', 'C5', 'A4', 'F#5', '4+9', 'C5', 'A4', '0+5', '4+9', 'C5', 'A4', 'E5', '4+9', 'C5', 'B4', '7', 'D5', 'E5', '4+9', 'C5', 'A4', '0+5', '4+9', 'C5', 'A4', 'F#5', '4+9', 'C5', 'A4', '0+5', '4+9', 'C5', 'A4', 'E5', '4+9', 'C5', 'B4', '7', 'D5', '9+0', '5', '4+7', '2+5', '7', '0+4', '11+2', '4', '0+4', 'E4', '2+5', '11', '11+2', '5', '7+11', '7', '9+0', '9', '11', '0', '4', 'A4', '5', 'F4', 'E5', 'A4', 'D5', '7', 'C5', 'A4', 'B4', '4', 'G4', 'C5', 'E4', 'D5', '11', 'G4', 'B4', '5', 'E4', 'G4', '7', 'E4', '4+9', '4+9', '4+9', '4+9', '4+9', '4+9', '2+7', '4+9', '4+9', '4+9', '4+9', '4+9', '2+7', 'E4', '4+9', 'A4', 'B4', 'C5', '4+9', 'B4', 'A4', 'E4', '4+9', 'C4', 'B3', '4+9', 'C4', 'A3', '4+9', 'C4', 'B3', '7', 'C4', 'D4', '5', 'E4', 'C4', '5', 'E4', 'D4', '4', 'E4', 'F4'

## Creating a MIDI file to play the music generated by the model

In [0]:
output_notes = []
offset = 0

for pattern in prediction_output:
    #if the pattern is a chord, then
    if ('+' in pattern) or pattern.isdigit():
        all_notes = pattern.split("+")
        temp_notes = []
        #print(all_notes)
        for current_note in all_notes:
          new_note = note.Note(int(current_note))
          new_note.storedInstrument = instrument.Piano()
          temp_notes.append(new_note)
        new_chord = chord.Chord(temp_notes) # It creates a chord with the help of the list of the notes
        new_chord.storedInstrument = instrument.Piano()
        output_notes.append(new_chord)
        
    else:
        #if the pattern is a note,then
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    offset += 0.5

In [141]:
print(output_notes)

[<music21.note.Note A>, <music21.chord.Chord C F>, <music21.note.Note C>, <music21.note.Note A>, <music21.note.Note E>, <music21.note.Note C>, <music21.note.Note B>, <music21.note.Note D>, <music21.note.Note E>, <music21.chord.Chord E A>, <music21.note.Note C>, <music21.note.Note A>, <music21.chord.Chord C F>, <music21.chord.Chord E A>, <music21.note.Note C>, <music21.note.Note A>, <music21.note.Note F#>, <music21.note.Note C>, <music21.note.Note A>, <music21.chord.Chord C F>, <music21.note.Note C>, <music21.note.Note A>, <music21.note.Note E>, <music21.note.Note E>, <music21.note.Note C>, <music21.note.Note B>, <music21.note.Note B>, <music21.note.Note C>, <music21.note.Note D>, <music21.note.Note G#>, <music21.note.Note E>, <music21.chord.Chord E A>, <music21.note.Note C>, <music21.note.Note A>, <music21.chord.Chord C F>, <music21.chord.Chord E A>, <music21.note.Note C>, <music21.note.Note A>, <music21.note.Note F#>, <music21.chord.Chord E A>, <music21.note.Note C>, <music21.note.Not

In [128]:
midi_stream = stream.Stream(output_notes)
midi_stream.write("midi","/content/drive/My Drive/MLplayground/output1.mid")

'/content/drive/My Drive/MLplayground/output1.mid'

In [129]:
midi_stream.show('midi')